## Import libraries

In [1]:
from os import link
import os
from unicodedata import name
import logging
import json
import scrapy
from scrapy.crawler import CrawlerProcess
import json
import pandas as pd

# import files csv

In [3]:
df_info = pd.read_csv("../working_files/games_info_prep.csv")
df_scores = pd.read_csv("../../scores_prep.csv")

In [15]:
display(df_info.columns)
df_scores.columns

Index(['Unnamed: 0', 'idgame', 'date', 'stade', 'location', 'attendance',
       'capacity', 'people', 'line', 'over_under', 'attendance_info',
       'dayofweek', 'dayofmonth', 'hour', 'month', 'minute'],
      dtype='object')

Index(['Unnamed: 0', 'season', 'week', 'awayteam', 'hometeam', 'awayscore',
       'homescore', 'idgame', 'score_abs', 'winner_home', 'winner_away'],
      dtype='object')

In [16]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231 entries, 0 to 1230
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       1231 non-null   int64 
 1   idgame           1231 non-null   int64 
 2   date             1231 non-null   object
 3   stade            1231 non-null   object
 4   location         1231 non-null   object
 5   attendance       1231 non-null   int64 
 6   capacity         1231 non-null   int64 
 7   people           1231 non-null   int64 
 8   line             493 non-null    object
 9   over_under       441 non-null    object
 10  attendance_info  1231 non-null   int64 
 11  dayofweek        1231 non-null   int64 
 12  dayofmonth       1231 non-null   int64 
 13  hour             1231 non-null   int64 
 14  month            1231 non-null   int64 
 15  minute           1231 non-null   int64 
dtypes: int64(11), object(5)
memory usage: 154.0+ KB


In [17]:
# merge files
df_scores_info = df_scores.merge(df_info, on = 'idgame')
display(df_scores_info)

,Unnamed: 0_x,season,week,awayteam,hometeam,awayscore,homescore,idgame,score_abs,winner_home,...,capacity,people,line,over_under,attendance_info,dayofweek,dayofmonth,hour,month,minute
0,0,2018,2,Ravens,Bengals,23,34,401030690,11,1,...,65515,76,NaN,NaN,1,4,14,0,9,20
1,1,2018,2,Panthers,Falcons,24,31,401030691,7,1,...,75000,97,NaN,NaN,1,6,16,17,9,0
2,2,2018,2,Chargers,Bills,31,20,401030704,-11,0,...,71621,97,NaN,NaN,1,6,16,17,9,0
3,3,2018,2,Texans,Titans,17,20,401030694,3,1,...,69143,90,NaN,NaN,1,6,16,17,9,0
4,4,2017,2,Texans,Bengals,13,9,400951620,-4,0,...,65515,81,NaN,NaN,1,4,15,0,9,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,1283,2021,18,Jets,Bills,10,27,401326587,17,1,...,71621,90,Line: BUF -16.0,Over/Under: 43.0,1,6,9,21,1,25
1222,1285,2021,18,Patriots,Dolphins,24,33,401326592,9,1,...,65326,100,Line: NE -6.0,Over/Under: 41.0,1,6,9,21,1,25
1223,1286,2021,18,Seahawks,Cardinals,38,30,401326597,-8,0,...,65000,98,Line: ARI -5.5,Over/Under: 48.5,1,6,9,21,1,25
1224,1287,2021,18,Panthers,Buccaneers,17,41,401326596,24,1,...,65618,100,Line: TB -11.0,Over/Under: 43.0,1,6,9,21,1,25


In [19]:
import datetime

In [29]:
df = df_scores_info.copy()

In [10]:
df = df_scores_info.copy()
# transform object in datetime
df['date'] = df['date'].apply(lambda x : datetime.datetime.fromisoformat(x))
# create a dataframe empty
df_time_teams = pd.DataFrame(columns=(['team','idgame','date',"delta_match"]))

teams = df["hometeam"].unique()

for team in teams:

    # work by home/away
    mask_home = df["hometeam"]==team
    mask_away = df["awayteam"]==team
    
    df_home = df.loc[mask_home,['hometeam','idgame','date']]
    df_away = df.loc[mask_away,['awayteam','idgame','date']]
    
    # rename column
    df_home.rename(columns={'hometeam':'team'}, inplace = True)
    df_away.rename(columns={'awayteam':'team'}, inplace = True)

    # concat away&home in one column team
    df_time = pd.concat([df_home,df_away])
    df_time.sort_values(by=['date'], ascending = True, inplace = True)

    # creat column
    df_time["delta_match"] =pd.Series()

    # calculate the time between two matches
    for index in range(len(df_time)):
        if index == 0:
            df_time["delta_match"].iloc[index] = 0
        else:
            df_time["delta_match"].iloc[index] = df_time["date"].iloc[index] - df_time["date"].iloc[index-1]

    # concat result
    df_time_teams = pd.concat([df_time_teams, df_time],axis=0, ignore_index= True)
    
     

C:\Users\sylva\AppData\Local\Temp\ipykernel_5340\2693333417.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_time["delta_match"] =pd.Series()
C:\Users\sylva\AppData\Local\Temp\ipykernel_5340\2693333417.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time["delta_match"].iloc[index] = 0
C:\Users\sylva\AppData\Local\Temp\ipykernel_5340\2693333417.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time["delta_match"].iloc[index] = df_time["date"].iloc[index] - df_time["date"].iloc[index-1

In [22]:
#Create working file

os.makedirs('../working_files', exist_ok=True)  
df_time_teams.to_csv('../working_files/games_timedelta.csv') 

In [11]:
df_time_teams.describe()

C:\Users\sylva\AppData\Local\Temp\ipykernel_5340\930267902.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_time_teams.describe()


,team,idgame,date,delta_match
count,2452,2452.0,2452,2452
unique,32,1226.0,519,252
top,Bills,400951572.0,2020-09-20 17:00:00+00:00,7 days 00:00:00
freq,80,2.0,20,689
first,NaN,NaN,2017-09-08 00:30:00+00:00,NaN
last,NaN,NaN,2022-01-10 01:20:00+00:00,NaN


In [66]:
len(df_time_teams)

2452

In [14]:
df_time_teams.tail(40)

,team,idgame,date,delta_match
2412,Chargers,400951650,2017-10-08 17:00:00+00:00,0.0
2413,Chargers,400951779,2017-10-15 20:25:00+00:00,7 days 03:25:00
2414,Chargers,400951715,2017-10-29 17:00:00+00:00,13 days 20:35:00
2415,Chargers,400951648,2017-11-12 18:00:00+00:00,14 days 01:00:00
2416,Chargers,400951573,2017-11-23 21:30:00+00:00,11 days 03:30:00
2417,Chargers,400951772,2017-12-17 01:25:00+00:00,23 days 03:55:00
2418,Chargers,400951642,2017-12-24 18:00:00+00:00,7 days 16:35:00
2419,Chargers,401030704,2018-09-16 17:00:00+00:00,265 days 23:00:00
2420,Chargers,401030798,2018-09-23 20:05:00+00:00,7 days 03:05:00
2421,Chargers,401030746,2018-10-14 17:00:00+00:00,20 days 20:55:00
